# Methoden-Workshop Machine Learning & AIA
1. **Inhaltsanalyse-Daten einlesen uns ansehen** mit der Python-Bibliothek für Data Management ``pandas``.
2. **Text in Features umwandeln** - mit ``sklearn``'s ``Vectorizer``.
3. **Classification Functions** auswählen.
4. Datensatz in **Train und Test Set** aufteilen.
5. Die Funktion auf den Trainingsdaten **fitten** aka. **trainieren**.
6. **Evaluieren** - Die Predictions mit der (manuellen) Codierung auf dem **Test Set abgleichen**.

Von: [anke.stoll@hhu.de](mailto:anke.stoll@hhu.de) <br>
Last edit: 25.02.2021

## 1. Von Excel nach Python und wieder zurück mit ``pandas``

In [48]:
import pandas as pd #Abkürzung für Faule und aus Konvention.

#### Daten einlesen - .csv or .txt oder Excel

Wo liegt/wie heißt der Datensatz?

In [49]:
df = pd.read_csv("Data Sets/HateSpeech_Tweets_DataSet.csv",
                 sep=",") #Ändern bei Excel auf ";")

#### Check out your data frame

In [50]:
df.head() #Zeigt die ersten 10 Zeilen.

,Tweet,HateSpeech
0,#Rosenmontag ist abgesagt. #Rapefugees also wi...,YES
1,bitte nicht die #Türkei zum #EU-Mitglied mache...,NO
2,Wieso bekommen #rapefugees mehr als unsere Har...,NO
3,Den verfluchten #Rapefugees den Krieg erklären...,YES
4,War das Wochenende im Ruhrpott unterwegs. Über...,YES


In [51]:
df.tail(10) #Oder die letzen 10.

,Tweet,HateSpeech
459,"Also ich finde, Europa ist sich einig wie noch...",NO
460,#dieanstalt #flüchtlinge und #Asylanten benöti...,YES
461,Die Regierung hat nichts im Griff. Das gibt ho...,NO
462,"Das gerechte in #Deutschland ist ja, dass nich...",NO
463,#radikaler #Imam in #Dänemark ruft z #Steinigu...,NO
464,Wo ist das Problem? Für Moslems ist es doch sc...,YES
465,Simone #Peter hat seit 7 Monaten und weit über...,NO
466,Bin gespannt auf #onebillionrising. Hoffentlic...,YES
467,#kipping ist so dumm! wenn #asylanten Wohnunge...,NO
468,Zukünft. #CDU Wähler aus #MEA marodieren auf M...,YES


In [52]:
len(df) #Anzahl der Zeilen.

469

In [53]:
df["HateSpeech"].value_counts() #Überblick zu den Werten in einer Spalte.

NO     315
YES    154
Name: HateSpeech, dtype: int64

## 2. Text zu Features - Unabhängigen Variablen programmieren

Die zweite super awesome Python Bibliothek heißt ``sklearn``. In dieser Bibliothek finden wir alles, was wir für ML brauchen. Auch das ML mit Text wird uns besonders einfach gemacht. In ``sklearn`` gibt es eine Unterabteilung ``feature_extraction.text``. Hier finden wir Funktionen, die uns das Umwandeln von Text zu Features (Unigramme oder N-Gramme) sehr einfach machen! Wir benutzen den ``CountVectorizer``. Full documentation auf der [sklearn Website](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

All vectorizers have several parameters that you can modify.

In [55]:
vec = CountVectorizer(ngram_range=(1,1)) #So wählen wir aus, dass wir nur Unigramme als Features wollen.
#Der Vectorizer entfernt außerdem Punctation und Sonderzeichen, sowie sehr kurze Wörter mit weniger als 2 Zeichen.
#Guckt euch die Dokumentation des Packages an, um noch mehr Möglichkeiten zu entdecken oder die Einstellungen zu verändern.

In [56]:
Tweets_vec = vec.fit_transform(df["Tweet"]) 
#Wir wenden den Vectorizer auf der Spalte "Tweet" in unserem Datensatz an.

In [57]:
Tweets_vec # 469 tweets und 2645 features (unigrams). #Sparse Matrix ist übrigens ein schlaues Format, das nicht die komplette Document Term Matrix abspeichert...
#...sondern nur die Zellen, die nicht 0 sind.

<469x2645 sparse matrix of type '<class 'numpy.int64'>'
	with 7461 stored elements in Compressed Sparse Row format>

#### Was sind unsere Features (UVs)?

In [58]:
vec.get_feature_names() #Das sind unsere Features

['00',
 '000',
 '000e',
 '01',
 '02',
 '10',
 '100',
 '1000',
 '100mal',
 '100t',
 '107',
 '12',
 '13',
 '14',
 '140',
 '16',
 '16a',
 '1920',
 '1h',
 '1k',
 '1live',
 '20',
 '2015',
 '2016',
 '2020',
 '21',
 '22',
 '24',
 '25',
 '250000',
 '28',
 '2g',
 '30',
 '300mio',
 '30t',
 '360',
 '39000',
 '40',
 '4000',
 '450',
 '50',
 '500',
 '500000',
 '60',
 '600m²',
 '60mio',
 '66',
 '70',
 '745',
 '90',
 'ab',
 'abartig',
 'abend',
 'abendlandes',
 'aber',
 'abferkeln',
 'abgefeiert',
 'abgehalten',
 'abgehen',
 'abgelehnt',
 'abgesagt',
 'abgeschoben',
 'ablehnen',
 'ablesen',
 'abläuft',
 'abmaasen',
 'abmerkeln',
 'abnehmen',
 'abreisenden',
 'absage',
 'abschafft',
 'abschaffung',
 'abschalten',
 'abschieben',
 'abschlagen',
 'abschließen',
 'absehbarer',
 'absurd',
 'aburteilen',
 'abwahl',
 'abwählen',
 'abzusagen',
 'achten',
 'afd',
 'afghanen',
 'afghanistan',
 'afrika',
 'aggressiver',
 'ahaa',
 'aka',
 'aktiv',
 'akzeptiert',
 'alaaf',
 'aldi',
 'algerien',
 'algerier',
 'ali',

Ich habe hier ein Stück Code eingefügt, dass ihr nutzen könnt, um euch die Häufigkeiten der einzelnen Features in eurem Datensatz anzusehen.

I used [this code from stuckoverflow](https://stackoverflow.com/questions/45805493/sorting-tfidfvectorizer-output-by-tf-idf-lowest-to-highest-and-vice-versa). Alternatively, you can try [this code](https://towardsdatascience.com/very-simple-python-script-for-extracting-most-common-words-from-a-story-1e3570d0b9d0) to get an overview of the most frequent words in your documents (before vectorization).

In [59]:
features = vec.get_feature_names()
sums = Tweets_vec.sum(axis=0) 

data = []

for col, term in enumerate(features):
    data.append( (term, sums[0,col] ))

df_ranks = pd.DataFrame(data, columns=['Feature','Count'])
df_ranks.sort_values('Count', ascending=False, inplace=True) 
#Setzt den Parameter ascending auf True und ihr erhaltet die Sortierung nach Count aufsteigend. 

In [60]:
df_ranks.head()

,Feature,Count
514,die,218
190,asylanten,203
1132,in,139
1763,rapefugees,120
2246,und,116


#### Probieren wir das ganze noch einmal, diesmal mit nur Bigrammen als Features!

In [61]:
vec = CountVectorizer(ngram_range=(2,2)) #Mit ngram_range=(1,2) erhaltet ihr Unigramme UND Bigramme als Features.

In [62]:
Tweets_vec = vec.fit_transform(df["Tweet"]) 

In [63]:
vec.get_feature_names() #Das sind unsere Features

['00 euro',
 '000 00',
 '000 000',
 '000 asylanten',
 '000 einheimischen',
 '000 euro',
 '000 fürs',
 '000 gekommen',
 '000 im',
 '000 neue',
 '000 syrer',
 '000e asylanten',
 '01 2016',
 '02 16',
 '02 2016',
 '10 mio',
 '10 prozent',
 '10 sachsen',
 '100 000',
 '100 000e',
 '100 merkel',
 '1000 anzeigen',
 '1000 dank',
 '1000 straftaten',
 '100mal gesagt',
 '100t aufgenommen',
 '107 männer',
 '12 mio',
 '13 das',
 '14 sicher',
 '140 zeichen',
 '16 02',
 '16 300mio',
 '16 bereicherung',
 '16 soeben',
 '16a nur',
 '1920 waren',
 '1h wie',
 '1k anstellen',
 '1live koelnhbf',
 '20 arbeitslos',
 '20 arbeitslose',
 '2015 am',
 '2015 in',
 '2015 rapefugees',
 '2015 war',
 '2016 2015',
 '2016 es',
 '2016 geld',
 '2016 islam',
 '2016 kamen',
 '2016 spd',
 '2016 und',
 '2020 zum',
 '21 01',
 '22 afd',
 '24 02',
 '25 kommen',
 '250000 die',
 '28 afd',
 '2g meth',
 '30 jahre',
 '30 prozent',
 '30 rapefugees',
 '300mio für',
 '30t asylanten',
 '360 mehr',
 '39000 asylanten',
 '40 60',
 '4000 chf',

## 3. Den Classifier programmieren

Wir wissen nun, wie man Features im Bag-of-Words-Style (Wörter mit ihren Häufigkeiten) erstellt. Als Kategorie (AV, Label, Class) haben wir in unserem Datensatz Hate Speech (YES/NO). Jetzt berechnen wir einen Zusammenhang, heißt, wir versuchen, die Kategorie _Hate Speech_ durch unsere Feautures zu schätzen (klassifizieren).

Als erstes müssen wir uns für eine Schätzfunktion - eine _Classification Function_ - entscheiden. Und zwar aus der Auswahl, die in der `sklearn` zu finden ist. Hier eine kleine Auswahl von Funktionen, die sich in der Forschung als geeignet herausgestellt haben. Jedoch muss man letztendlich durch Trail und Error herausfinden, welche sich am besteb eignet.

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [65]:
model = MultinomialNB() 
#Ich entscheide mich für die schnelle Variante, mit Default Parameter-Einstellungen (daher ist die Klammer () leer).

#### Den Datensatz in Train und Test Set aufteilen

Als ersten teilen wir den Datensatz in Trainingsdaten und Testdaten auf.

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X = df["Tweet"]

In [68]:
y =df["HateSpeech"]

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, #X is your pandas column with the documents.
                                                    y, #y is your pandas coulmn with category.
                                                    test_size=0.33,#Relative size of the test set. e.g. 33% 
                                                    random_state=42) #Chose a number to reproduce the split.

Jetzt wird der Text in X in features umgewandlet (vektorisiert), wie wir es weiter oben gelernt haben. Achte darauf, zunächst nur die Texte im Train Set zu vektorisieren!

In [70]:
vec = CountVectorizer(ngram_range=(1,1)) #Ich nehme nur Unigramme.

In [71]:
X_train_vec = vec.fit_transform(X_train) #Do this ONLY ON X TRAIN !

In [72]:
X_train_vec #314 Kommentare und 1987 Features (Unigramme hier)

<314x1987 sparse matrix of type '<class 'numpy.int64'>'
	with 4969 stored elements in Compressed Sparse Row format>

Jetzt wird der Classifier trainiert! Das ist genau eine Zeile Code! Und geht mit unserem kleinen Train Set und Naive Bayes in Sekundenschnelle!

In [77]:
model.fit(X_train_vec, y_train) #Fit the model, what means training. 

MultinomialNB()

Jetzt lassen wir unser trainiertes (gefittetes) Modell Predictions auf dem Test Set machen, damit wir die klassifizierten Werte mit den tatsächlichen Labels vergleichen können.

Vorher müssen die Texte im Test Set ebenfalls in Features transformiert werden. Nutze dafür nur die Funktion transform (nicht fit_transform) deines Vectorizers.

In [78]:
X_test_vec = vec.transform(X_test)# Mache Predictions auf dem Test Set für die Evaluation. 
#Benutze die funtion transform, nicht fit_transform!

In [75]:
y_pred = model.predict(X_test_vec) #predict ist die Funktion, mit der die Predictions gemacht werden.
#Diese werden im Objekt y_pred abgespeichert.

In [76]:
y_pred #Werfen wir schon mal einen Blick auf die klassifizierten Werte...Scheint jedenfalls geklappt zu haben.

array(['YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'YES', 'YES', 'NO', 'NO', 'NO', 'YES', 'NO', 'YES', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES',
       'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'YES', 'YES', 'NO',
       'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES', 'YES',
       'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO',
       'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'N

## Evaluation!

Nun vergleichen wir die Ergebnisse des Classifiers mit den Labels auf dem Test Set. Hierfür importieren wir die ein paar Maße und Funktionen aus `sklear.metrics`, die wir für die Evaluation nutzen.

In [79]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [80]:
#This is a nice overview of the performance of your model in all categories (on the test set).
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          NO       0.68      0.94      0.79        99
         YES       0.68      0.23      0.35        56

    accuracy                           0.68       155
   macro avg       0.68      0.59      0.57       155
weighted avg       0.68      0.68      0.63       155



In [82]:
#Nice confusion matrix output as a pandas data frame.
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted NO', 'Predicted YES'],
    index=['True NO', 'True YES']
)

,Predicted NO,Predicted YES
True NO,93,6
True YES,43,13


## Fertig!